# Term Project Spring 2020
## Team Game Cancelled


talk about project

---
## ERD for CSV Data
The database should not leave off any files or columns.
Normalize to at least BCNF.

year
    semester(term)
        courses.csv
        course_meetings.csv
        course catalog.csv

### Imports

In [1]:
%load_ext sql
import pandas as pd
import sqlite3

![Course Data ERD](James/files/CourseDataERD.png)

In [2]:
# Switch the relationship between course catalogs and course offerings

---
## Data Dictionary
markdown tables instead of list(s)

[Course Data Dictionary](Docs/CourseDataDictionary.md)

---
## Set Up - SQL and Database

In [3]:
%sql sqlite:///CourseData.db
conn = sqlite3.connect('CourseData.db')

# 1. Create Tables From ERD

In [4]:
%%sql

-- Course Offerings table
DROP TABLE IF EXISTS COURSE_OFFERINGS;
CREATE TABLE COURSE_OFFERINGS (
    CourseID INTEGER PRIMARY KEY,
    Term TEXT NOT NULL,
    CRN INTEGER NOT NULL,
    CatalogID TEXT NOT NULL,
    Section TEXT NOT NULL,
    Credits TEXT NOT NULL,
    Title TEXT NOT NULL,
    Timecodes TEXT NOT NULL,
    PrimaryInstructorID TEXT NOT NULL,
    Capacity INTEGER NOT NULL,
    Actual INTEGER NOT NULL,
    Remaining INTEGER NOT NULL
);

-- Course Catalogs table
DROP TABLE IF EXISTS COURSE_CATALOGS;
CREATE TABLE COURSE_CATALOGS(
    CourseID INTEGER PRIMARY KEY,
    CatalogYear TEXT NOT NULL,
    CatalogID TEXT NOT NULL,
    ProgramID TEXT NOT NULL,
    CourseTitle TEXT NOT NULL,
    Credits TEXT NOT NULL,
    Prequisites TEXT,
    Corequisites TEXT,
    Fees TEXT,
    Attributes TEXT,
    Description TEXT
);

-- Programs table
DROP TABLE IF EXISTS PROGRAMS;
CREATE TABLE PROGRAMS (
    ProgramID INTEGER PRIMARY KEY,
    ProgramCode TEXT NOT NULL,
    ProgramName TEXT NOT NULL
);

-- Instructors table
DROP TABLE IF EXISTS INSTRUCTORS;
CREATE TABLE INSTRUCTORS (
    InstructorID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL
);

-- Course Meetings table
DROP TABLE IF EXISTS COURSE_MEETINGS;
CREATE TABLE COURSE_MEETINGS (
    CourseMeetingID INTEGER PRIMARY KEY,
    CRN INTEGER NOT NULL,
    LocationID TEXT NOT NULL,
    Day TEXT NOT NULL,
    StartDateTime TEXT NOT NULL,
    EndDateTime TEXT NOT NULL
);

-- Location table
DROP TABLE IF EXISTS LOCATION;
CREATE TABLE LOCATION (
    LocationID TEXT PRIMARY KEY
);

 * sqlite:///CourseData.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

---
# 2. Importing CSV Data

### List of CSVs:
(Catalogs)
    CourseCatalog2017_2018.csv | https://ba-lab.fairfield.edu/user/jneri/lab/tree/term-project-teamgamecancelled/SourceData/Catalogs/CourseCatalog2017_2018.csv
    CourseCatalog2018_2019.csv
(Each Semester's Folder)
    course_meetings.csv
    courses.csv

In [5]:
catalogyear = ['2017_2018', '2018_2019']

for catyear in catalogyear:
    filepath = 'SourceData/Catalogs/CourseCatalog'+catyear+'.csv'
    data = pd.read_csv(filepath)
    data['catyear']=catyear
    data.to_sql('IMPORT_COURSE_CATALOGS', conn, if_exists='append', index=False)

In [6]:
terms = ['Fall2014','Fall2015','Fall2016','Fall2017','Fall2018',
         'Spring2015','Spring2016','Spring2017','Spring2018','Spring2019',
         'SpringBreak2017',
         'Summer2015','Summer2016','Summer2017','Summer2018',
         'Winter2015','Winter2016','Winter2017','Winter2018']

for term in terms:
    filepath = 'SourceData/'+term+'/courses.csv'
    data = pd.read_csv(filepath)
    data.to_sql('IMPORT_COURSE_OFFERINGS',conn,if_exists='append',index=False) 
    
    filepath = 'SourceData/'+term+'/course_meetings.csv'
    data = pd.read_csv(filepath)
    data.to_sql('IMPORT_COURSE_MEETINGS',conn,if_exists='append',index=False)

In [7]:
%%sql
-- Record Counts for Course Offerings
SELECT 
    (SELECT Count(*) FROM IMPORT_COURSE_OFFERINGS) as 'RawCount',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM IMPORT_COURSE_OFFERINGS)) as 'DistinctCount';

 * sqlite:///CourseData.db
Done.


RawCount,DistinctCount
15937,15937


In [8]:
%%sql
-- Record Counts for Catalog Courses
SELECT 
    (SELECT Count(*) FROM IMPORT_COURSE_CATALOGS) as 'RawCount',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM IMPORT_COURSE_CATALOGS)) as 'DistinctCount';

 * sqlite:///CourseData.db
Done.


RawCount,DistinctCount
4440,4440


In [9]:
%%sql
-- Record Counts for Course Meetings
SELECT 
    (SELECT Count(*) FROM IMPORT_COURSE_MEETINGS) as 'RawCount',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM IMPORT_COURSE_MEETINGS)) as 'DistinctCount';

 * sqlite:///CourseData.db
Done.


RawCount,DistinctCount
284907,284847


In [10]:
# Note: 60 Duplicates -- Will look into later

# 3 & 4 Transform and Load Data into ERD

In [11]:
# Fall 2014 - 987

In [12]:
%%sql

DELETE FROM PROGRAMS;

INSERT INTO PROGRAMS (ProgramCode,ProgramName)
SELECT DISTINCT program_code,program_name 
FROM IMPORT_COURSE_CATALOGS
ORDER BY program_code;

SELECT * FROM PROGRAMS
LIMIT 10

 * sqlite:///CourseData.db
0 rows affected.
83 rows affected.
Done.


ProgramID,ProgramCode,ProgramName
1,AC,Accounting
2,AE,Applied Ethics
3,AH,Art History
4,AN,Asian Studies
5,AR,Arabic
6,AS,American Studies
7,AY,Anthropology
8,BB,Business
9,BEN,Bioengineering
10,BI,Biology


In [13]:
%%sql

DELETE FROM INSTRUCTORS;

INSERT INTO INSTRUCTORS (Name)
SELECT DISTINCT primary_instructor
FROM IMPORT_COURSE_OFFERINGS 
WHERE primary_instructor <> 'TBA' AND primary_instructor NOT LIKE '%/%';

SELECT * FROM INSTRUCTORS
LIMIT 10

 * sqlite:///CourseData.db
0 rows affected.
1095 rows affected.
Done.


InstructorID,Name
1,Michael P. Coyne
2,Rebecca I. Bloch
3,Paul Caster
4,Jo Ann Drusbosky
5,Arleen N. Kardos
6,Scott M Brenner
7,Kevin C. Cassidy
8,Bruce Bradford
9,Milo W. Peck
10,Stephen E. Yost


In [14]:
%%sql

DELETE FROM LOCATION;

INSERT INTO LOCATION (LocationID)
SELECT DISTINCT location
FROM IMPORT_COURSE_MEETINGS
ORDER BY location;

SELECT * FROM LOCATION
LIMIT 10

 * sqlite:///CourseData.db
0 rows affected.
207 rows affected.
Done.


LocationID
BCC 200
BD
BH
BH BY ARR
BLM 112
BLM LL105
BNW 124
BNW 127
BNW 128
BNW 129B


In [15]:
%%sql 

DELETE FROM COURSE_CATALOGS;

INSERT INTO COURSE_CATALOGS (CatalogYear,CatalogID,ProgramID,CourseTitle,Credits,Prequisites,Corequisites,Fees,Attributes,Description)
SELECT DISTINCT catyear,catalog_id,ProgramID,course_title,credits,prereqs,coreqs,fees,attributes,description
FROM IMPORT_COURSE_CATALOGS 
    JOIN PROGRAMS ON (program_code = ProgramCode);
    
SELECT * FROM COURSE_CATALOGS
LIMIT 5

 * sqlite:///CourseData.db
0 rows affected.
4440 rows affected.
Done.


CourseID,CatalogYear,CatalogID,ProgramID,CourseTitle,Credits,Prequisites,Corequisites,Fees,Attributes,Description
1,2017_2018,AN 0301,4,Independent Study,1-3 Credits,None,None,None,None,Students undertake an individualized program of study in consultation with a director from the Asian studies faculty.
2,2017_2018,AN 0310,4,Asian Studies Seminar,3 Credits,None,None,None,None,"This seminar examines selected topics concerning Asia. This course is taught in conjunction with another 100-300 level course from a rotation of course offerings. Consult the Asian Studies director to identify the conjoined course for a given semester. The seminar concentrates on topics within the parameters of the conjoined course syllabus but adds research emphasis. Students registered for this course must complete a research project, to include 300-level research, in addition to the regular research requirements of the conjoined course, and a 25-50 page term paper in substitution of some portion of the conjoined course requirements, as determined by the instructor. Open to juniors and seniors only."
3,2017_2018,BU 0211,12,Legal Environment of Business,3 Credits,Junior standing.,None,None,None,"This course examines the broad philosophical as well as practical nature and function of the legal system, and introduces students to the legal and social responsibilities of business. The course includes an introduction to the legal system, the federal courts, Constitutional law, the United States Supreme Court, the civil process, and regulatory areas such as employment discrimination, protection of the environment, and corporate governance and securities markets."
4,2017_2018,BU 0220,12,Environmental Law and Policy,3 Credits,None,None,None,"EVME Environmental Studies Major Elective, EVPE Environmental Studies Elective, EVSS Environmental Studies: Social Science, MGEL Management: General Elective","This course surveys issues arising out of federal laws designed to protect the environment and manage resources. It considers in detail the role of the Environmental Protection Agency in the enforcement of environmental policies arising out of such laws as the National Environmental Policy Act, the Clean Water Act, and the Clear Air Act, among others. The course also considers the impact of Congress, political parties, bureaucracy, and interest groups in shaping environmental policy, giving special attention to the impact of environmental regulation on business and private property rights."
5,2017_2018,BU 0311,12,"The Law of Contracts, Sales, and Property",3 Credits,BU 0211.,None,None,None,"This course examines the components of common law contracts including the concepts of offer and acceptance, consideration, capacity and legality, assignment of rights and delegation of duties, as well as discharge of contracts. The course covers Articles 2 and 2A of the Uniform Commercial Code relating to leases, sales of goods, and warranties. The course also considers personal and real property, and bailments."


In [19]:
%%sql 

DELETE FROM COURSE_MEETINGS;

INSERT INTO COURSE_MEETINGS (CRN,LocationID,Day,StartDateTime,EndDateTime)
SELECT DISTINCT crn,location,day,start,end
FROM IMPORT_COURSE_MEETINGS 
    LEFT JOIN LOCATION ON (location = LocationID);
    
SELECT * FROM COURSE_MEETINGS
LIMIT 5

 * sqlite:///CourseData.db
0 rows affected.
284847 rows affected.
Done.


CourseMeetingID,CRN,LocationID,Day,StartDateTime,EndDateTime
1,70384,DSB 105,T,2014-09-02T08:00:00,2014-09-02T09:15:00
2,70384,DSB 105,F,2014-09-05T08:00:00,2014-09-05T09:15:00
3,70384,DSB 105,T,2014-09-09T08:00:00,2014-09-09T09:15:00
4,70384,DSB 105,F,2014-09-12T08:00:00,2014-09-12T09:15:00
5,70384,DSB 105,T,2014-09-16T08:00:00,2014-09-16T09:15:00


In [31]:
%%sql 

DELETE FROM COURSE_OFFERINGS;

INSERT INTO COURSE_OFFERINGS (Term,CRN,Credits,CatalogID,Section,Title,Timecodes,PrimaryInstructorID,Capacity,Actual,Remaining)
SELECT DISTINCT term,crn,IMPORT_COURSE_OFFERINGS.credits,catalog_id,section,title,timecodes,primary_instructor,cap,act,rem
FROM IMPORT_COURSE_OFFERINGS 
    LEFT JOIN COURSE_CATALOGS ON (catalog_id = CatalogID)
    LEFT JOIN INSTRUCTORS ON (primary_instructor = Name)
    LEFT JOIN COURSE_MEETINGS USING (CRN);
    
SELECT * FROM COURSE_OFFERINGS
LIMIT 5

 * sqlite:///CourseData.db
0 rows affected.
15937 rows affected.
Done.


CourseID,Term,CRN,CatalogID,Section,Credits,Title,Timecodes,PrimaryInstructorID,Capacity,Actual,Remaining
1,Fall2014,70384,AC 0011,C01,3.000,Introduction to Financial Accounting,['TF 0800am-0915am 09/02-12/08 DSB 105'],Michael P. Coyne,0,31,-31
2,Fall2014,70385,AC 0011,C02,3.000,Introduction to Financial Accounting,['TF 0930am-1045am 09/02-12/08 DSB 105'],Michael P. Coyne,0,31,-31
3,Fall2014,70382,AC 0011,C03,3.000,Introduction to Financial Accounting,['TF 1230pm-0145pm 09/02-12/08 DSB 105'],Michael P. Coyne,0,31,-31
4,Fall2014,70291,AC 0011,C04,3.000,Introduction to Financial Accounting,['MR 1100am-1215pm 09/02-12/08 DSB 111'],Rebecca I. Bloch,0,29,-29
5,Fall2014,70350,AC 0011,C05,3.000,Introduction to Financial Accounting,['MR 1230pm-0145pm 09/02-12/08 DSB 111'],Rebecca I. Bloch,0,30,-30


# 5. Check

# 6. Empty out Tables to Reclaim Storage